<a href="https://colab.research.google.com/github/ozturkcemal/CorkPubCrawl/blob/main/CorkPubCrawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ortools openrouteservice

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [2]:
# Step 1: Import libraries
import openrouteservice
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [22]:
# Step 2: Setup your coordinates and API key
coordinates = [
    [-8.4756, 51.8985],  # Cork
    [-6.2603, 53.3498],  # Dublin
    [-8.6306, 52.6675],  # Limerick
    [-8.9563, 51.9036],  # Macroom
    [-7.1870, 52.2544],  # Waterford
]

In [23]:
api_key = "5b3ce3597851110001cf6248ac6b2739185944338284b13037a0ba41"
client = openrouteservice.Client(key=api_key)


In [24]:
# Step 3: Get the distance matrix from ORS
matrix = client.distance_matrix(
    locations=coordinates,
    profile='driving-car',
    metrics=['distance'],
    units='m'
)
distance_matrix = matrix['distances']

In [25]:
# Step 4: Create the data model
def create_data_model():
    data = {}
    data['distance_matrix'] = distance_matrix
    data['num_vehicles'] = 1
    data['depot'] = 0  # Starting point
    return data

data = create_data_model()

print("Distance Matrix:")
for row in distance_matrix:
    print(row)

Distance Matrix:
[0.0, 250375.11, 99402.8, 42178.38, 118048.88]
[259431.5, 0.0, 198072.84, 293861.78, 166172.17]
[99255.61, 199256.67, 0.0, 110158.95, 129429.84]
[42009.57, 293068.19, 109801.24, 0.0, 156680.06]
[121651.19, 165257.66, 128178.52, 156081.45, 0.0]


In [26]:
# Step 5: OR-Tools setup
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'],
                                       data['depot'])
routing = pywrapcp.RoutingModel(manager)

def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(data['distance_matrix'][from_node][to_node])

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [27]:
# Step 6: Solve and print the solution
solution = routing.SolveWithParameters(search_parameters)

def print_solution(manager, routing, solution):
    print("Optimized route:")
    index = routing.Start(0)
    route = []
    route_distance = 0
    while not routing.IsEnd(index):
        node = manager.IndexToNode(index)
        route.append(node)
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    route.append(manager.IndexToNode(index))
    print(" -> ".join(str(i) for i in route))
    print("Total distance: {:.2f} km".format(route_distance / 1000))

if solution:
    print_solution(manager, routing, solution)
else:
    print("No solution found.")

Optimized route:
0 -> 4 -> 1 -> 2 -> 3 -> 0
Total distance: 633.54 km
